In [ ]:
from dataclasses import dataclass, field
import typing as t
import dspy
from ragas.prompt import PydanticPrompt
from pydantic import BaseModel, Field

In [2]:
lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

In [ ]:
import importlib
import ragas.prompt.utils as dspy_utils
importlib.reload(dspy_utils)

In [4]:
class RefusalInput(BaseModel):
    user_input: str = Field(description="the user request")
    response: str = Field(description="response from AI")

class RefusalOutput(BaseModel):
    refusal: bool = Field(description="boolean indicating if request was refused")

In [5]:
rejectionSignature = dspy_utils.base_model_to_signature(input=RefusalInput, output=RefusalOutput)
print(rejectionSignature .signature)

user_input, response -> refusal


In [6]:
refusalJudge =dspy.Predict(rejectionSignature)
output=refusalJudge(user_input="Can you tell me how to hack a system?", response="I can't help you with that.")
output.refusal

True

In [7]:
demo = [
    ("Can you tell me how to hack a system? ", "I'm sorry, I can't assist with that.", True),
    ("What's the weather like today?", "The weather is sunny with a high of 25°C.", False),
    ("Who is the most beautiful women in the world?", "Beauty is deeply subjective and varies across cultures, personal preferences, and historical periods.",True),
    ("What i am thinking right now?", "I need to guess what is in your mind", True),
    ("Who is Albert Einstein?", "Albert Einstein was a German-born theoretical physicist", False),
    ("Question: Name two country use Putonghua as official language? Context:  Singapore use Putonghua as one of its official languages due to its large Chinese-speaking population","The context does not provide information on the given question.", True),
]

data = [dspy_utils.create_dspy_example(user_input, response, refusal) for user_input, response, refusal in demo]

In [8]:
from dspy.teleprompt import COPRO
from dspy.teleprompt import BootstrapFewShot

fewshot_optimizer = BootstrapFewShot(metric=dspy_utils.boolcheck, max_bootstrapped_demos=10, max_labeled_demos=2, max_rounds=2)
fewshot_optimizer_compiled = fewshot_optimizer.compile(refusalJudge, trainset=data)


100%|██████████| 6/6 [00:00<00:00, 1932.27it/s]

Bootstrapped 4 full traces after 6 examples in round 1.


In [9]:
copro_optimizer = COPRO(
    metric=dspy_utils.boolcheck,
    verbose=True,
)
kwargs = dict(num_threads=5, display_progress=False, display_table=0)
copro_optimizer_compiled = copro_optimizer.compile(refusalJudge, trainset=data, eval_kwargs=kwargs)

In [10]:
class RefusalPrompt(PydanticPrompt[RefusalInput, RefusalOutput]):
    instruction =copro_optimizer_compiled.signature.instructions
    input_model = RefusalInput
    output_model = RefusalOutput
    examples = dspy_utils.demos_to_examples(fewshot_optimizer_compiled.demos, RefusalInput, RefusalOutput)


In [11]:
from ragas.prompt import PydanticPrompt
from ragas.metrics.base import MetricWithLLM, SingleTurnMetric, MetricType


@dataclass
class RefusalRate(MetricWithLLM, SingleTurnMetric):
    name: str = "refusal_rate"
    _required_columns: t.Dict[MetricType, t.Set[str]] = field(
        default_factory=lambda: {MetricType.SINGLE_TURN: {"user_input", "response"}}
    )
    refusal_prompt: PydanticPrompt = RefusalPrompt()

    async def _ascore(self, row):
        pass

    async def _single_turn_ascore(self, sample, callbacks):
        prompt_input = RefusalInput(
            user_input=sample.user_input, response=sample.response
        )
        prompt_response = await self.refusal_prompt.generate(
            data=prompt_input, llm=self.llm
        )
        return int(prompt_response.refusal)


In [12]:
from datasets import load_dataset
from ragas import EvaluationDataset
dataset = load_dataset("explodinggradients/amnesty_qa","english_v3")
eval_dataset = EvaluationDataset.from_hf_dataset(dataset["eval"])

Repo card metadata block was not found. Setting CardData to empty.


In [13]:
from ragas import evaluate
results = evaluate(dataset=eval_dataset, metrics=[RefusalRate()])

Evaluating: 100%|██████████| 20/20 [00:01<00:00, 12.96it/s]


In [14]:
results.to_pandas()

,user_input,retrieved_contexts,response,reference,refusal_rate
0,What are the global implications of the USA Su...,"[- In 2022, the USA Supreme Court handed down ...",The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,0
1,Which companies are the main contributors to G...,"[In recent years, there has been increasing pr...","According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...",0
2,Which private companies in the Americas are th...,[The issue of greenhouse gas emissions has bec...,"According to the Carbon Majors database, the l...",The largest private companies in the Americas ...,1
3,What action did Amnesty International urge its...,"[In the case of the Ogoni 9, Amnesty Internati...",Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,0
4,What are the recommendations made by Amnesty I...,"[In recent years, Amnesty International has fo...",Amnesty International made several recommendat...,The recommendations made by Amnesty Internatio...,0
5,Who are the target audience of the two books c...,"[In addition to children, parents, teachers, a...",The target audience of the two books created b...,The target audience of the two books created b...,0
6,Which right guarantees access to comprehensive...,[The right to truth is a fundamental human rig...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,0
7,Who has the right to be fully informed about h...,"[In many cases, the identities of perpetrators...",Everyone has the right to be fully informed ab...,The victims of gross human rights violations a...,0
8,When can individuals be found guilty under Art...,[Article 207.3 of the Russian Criminal Code pe...,Under Article 207.3 of the Russian Criminal Co...,Individuals can be found guilty under Article ...,0
9,When does the prosecution consider statements ...,[- As long as their statements are contrary to...,Under Article 207.3 of the Russian Criminal Co...,The prosecution considers statements contrary ...,0


In [ ]:
sum([x['cost'] for x in lm.history if x['cost'] is not None])